# Modelo DOC: 

## Preprocesamiento held out

In [1]:
pwd

'/Users/fernandomarquez/Documents/Fleni/Investigacion/Modelo_EMCS/Modelo3_mes1'

In [2]:
#Carga de las librerías y datasets utilizados
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import sklearn

df= pd.read_excel('/Users/fernandomarquez/Documents/Fleni/DF/vero_completado.xlsx',
                  parse_dates=['Fecha de ingreso', 'Fecha de alta del prote',
                  'Fecha de alta  internación','Fecha de la lesión'])

/Users/fernandomarquez/.local/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Dropna

In [3]:
df=df.dropna(subset=['CRSe_T'])

In [4]:
df.EMCS.value_counts()

0.0    9
1.0    8
Name: EMCS, dtype: int64

## Columnas

In [5]:
#Eliminar columnas que no son numericas
cols_drop=['Agregados','Año estadistica', 
           'Apellido y Nombre',
           'Observaciones',
       'Finalización tratamiento: SI - NO',
       'Motivo no finalizacion: obito o derivacion sin reingreso',
       'Destino al finalizar: CR - arriba (cronicos)- externacion',
       'Derivacion SI-NO', 'Tipo de diagnóstico',
       'Tiempo de evolución al ingreso (días) (autom.)',
       'Cantidad días del programa (autom.)',
       'EMCS_uso_objetos ', 'EMCS_comunicacion', 'FIM Inicio', 'FIM Alta',
       'Unnamed: 44', 'Unnamed: 49', 'Unnamed: 50',
       'excluidos'
          ]
#Dropeo las columnas que no son numericas
df = df.drop(cols_drop, 1)

In [6]:
dict_cols= {'Nro HC':'NHC',
            'Apellido y Nombre':'nya',
            'Fecha de ingreso':'ingreso',
            'Fecha de alta del prote':'alta_prote',
            'Fecha de alta  internación':'alta',
            'Edad':'edad', 
            'Sexo':'genero',
            'Diagnóstico (TEC, ANOXIA, OTROS)':'etiologia', 
            'Fecha de la lesión':'lesion',
            'Estado de conciencia al  Alta emergencia con uno o dos crietrios ?':'conciencia_e',
            'SVSR/MC Inicial':'conciencia_i',
            'EMCS_uso_objetos ':'EMCS_uso_objetos', 'EMCS_comunicacion':'EMCS_comunicacion',
           'EMCS':'EMERGENCIA'}
#Renombrado de columnas
df=df.rename(columns=dict_cols)
df.columns

Index(['NHC', 'ingreso', 'alta_prote', 'alta', 'edad', 'genero', 'etiologia',
       'lesion', 'CRSi_A', 'CRSi_V', 'CRSi_M', 'CRSi_Ve', 'CRSi_C', 'CRSi_Ar',
       'CRSi_T', 'conciencia_i', 'JFK_T', 'JFK_A', 'JFK_V', 'JFK_M', 'JFK_Ve',
       'JFK_C', 'JFK_Ar', 'EMERGENCIA', 'JFK_fecha', 'CRSe_A', 'CRSe_V',
       'CRSe_M', 'CRSe_Ve', 'CRSe_C', 'CRSe_Ar', 'CRSe_T', 'conciencia_e'],
      dtype='object')

In [7]:
def draw_missing_data_table(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data
draw_missing_data_table(df)

,Total,Percent
conciencia_e,0,0.0
conciencia_i,0,0.0
ingreso,0,0.0
alta_prote,0,0.0
alta,0,0.0
edad,0,0.0
genero,0,0.0
etiologia,0,0.0
lesion,0,0.0
CRSi_A,0,0.0


### Excluir los que emergieron al ingreso

In [8]:
df=df.loc[df.conciencia_i != 'EMCS']

In [9]:
def autoconvert_datetime(value):
    formats = ['%m/%d/%Y', '%m-%d-%y']  # formats to try
    result_format = '%d-%m-%Y'  # output format
    for dt_format in formats:
        try:
            dt_obj = datetime.strptime(value, dt_format)
            return dt_obj.strftime(result_format)
        except Exception as e:  # throws exception when format doesn't match
            pass
    return value  # let it be if it doesn't match

df['ingreso'] = df['ingreso'].apply(autoconvert_datetime)
df['alta_prote'] = df['alta_prote'].apply(autoconvert_datetime)
df['alta'] = df['alta'].apply(autoconvert_datetime)

In [10]:
df.dtypes

NHC                    float64
ingreso         datetime64[ns]
alta_prote              object
alta                    object
edad                   float64
genero                  object
etiologia               object
lesion          datetime64[ns]
CRSi_A                 float64
CRSi_V                 float64
CRSi_M                 float64
CRSi_Ve                float64
CRSi_C                 float64
CRSi_Ar                float64
CRSi_T                 float64
conciencia_i            object
JFK_T                  float64
JFK_A                  float64
JFK_V                  float64
JFK_M                  float64
JFK_Ve                 float64
JFK_C                  float64
JFK_Ar                 float64
EMERGENCIA             float64
JFK_fecha       datetime64[ns]
CRSe_A                  object
CRSe_V                 float64
CRSe_M                 float64
CRSe_Ve                float64
CRSe_C                 float64
CRSe_Ar                float64
CRSe_T                 float64
concienc

# Mapping

In [11]:
dic_conciencia_e={'SVSR':'SVSR', 
                  'SMC':'EMCS', 
                  'MC':'MC', 
                  'EMC':'EMCS',
                  'EMCS':'EMCS',
                  'MC ':'MC'}
df['conciencia_e']=df['conciencia_e'].map(dic_conciencia_e)

In [12]:
#Agrego columna de edad mapeada con 1 por defecto
df['edad_map']=1
#Cambio el 1 por 0 en los pacientes menores de 30 años
df.loc[(df['edad'] < 30), 'edad_map']=0

#Agrego columna de edad mapeada con 1 por defecto
df['edad_map1']=1
#Cambio el 1 por 0 en los pacientes menores de 50 años
df.loc[(df['edad'] < 50), 'edad_map1']=0

In [13]:
from datetime import datetime, timedelta

df['alta_prote']=df['alta_prote'].values.astype('datetime64[D]')
df['LOS']=(df['alta_prote'].values)-(df['ingreso'])
df['LOS']=df['LOS']/ timedelta (days=1)
df.LOS.describe()

count     16.000000
mean     183.000000
std      172.714794
min       11.000000
25%       68.500000
50%      125.500000
75%      241.250000
max      575.000000
Name: LOS, dtype: float64

# Criterios de exclusion

In [14]:
print('Cantidad de pacientes TOTAL')
print(df.shape)

print('Cantidad de pacientes que estuvieron internados más de 1 MES')
print(df[(df['LOS'] > 30)].shape)
df[(df['LOS'] > 30)].groupby(by='EMERGENCIA').count()

Cantidad de pacientes TOTAL
(16, 36)
Cantidad de pacientes que estuvieron internados más de 1 MES
(13, 36)


,NHC,ingreso,alta_prote,alta,edad,genero,etiologia,lesion,CRSi_A,CRSi_V,...,CRSe_V,CRSe_M,CRSe_Ve,CRSe_C,CRSe_Ar,CRSe_T,conciencia_e,edad_map,edad_map1,LOS
EMERGENCIA,,,,,,,,,,,,,,,,,,,,,
0.0,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
1.0,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


### Elimino los que tieneen menos de 30 días de seguimiento

In [15]:
df=df.loc[(df['LOS'] > 30)]
df.LOS.describe()

count     13.000000
mean     221.384615
std      169.605689
min       58.000000
25%       90.000000
50%      173.000000
75%      281.000000
max      575.000000
Name: LOS, dtype: float64

### Conversion columna CRSe_A object a float64

In [16]:
df['CRSe_A']=df['CRSe_A'].astype(str).astype(int)

### Creacion columns TIEMPO EVOLUCION

In [17]:
df['evolucion']=df['ingreso']-df['lesion'] 
#Convertir variable timedelta en días a numero
df['evolucion']=df['evolucion']/ timedelta (days=1)

In [18]:
# DF de variables categoricas
df_cat=df[['genero', 'etiologia','conciencia_i','conciencia_e', 'EMERGENCIA', 'edad_map','edad_map1']]
for column in df_cat.columns: 
    print(df_cat[column].unique())

['M' 'F']
['Otros: ACV' 'ACV' 'TEC' 'Anoxia']
['MC' 'SVSR']
['EMCS' 'SVSR' 'MC']
[1. 0.]
[1 0]
[0 1]


In [19]:
dic = {'TEC':'TEC', 'tec':'TEC', 'TEC  ':'TEC', 'ACV':'ACV', 'Anoxia':'Anoxia', 'TEC ':'TEC', 'ANOXIA':'Anoxia', 
       'anoxia':'Anoxia','Tumor':'Tumor', 'Anoxia ':'Anoxia', ' ACV':'ACV', 'Otros: ACV':'ACV'}

df['etiologia']=df['etiologia'].map(dic)
df.etiologia.unique()

array(['ACV', 'TEC', 'Anoxia'], dtype=object)

# Nuevas variables

In [20]:
df['seguimiento']=(df['JFK_fecha']-df['ingreso'])/ timedelta (days=1)

# Creacion columna Varición CRS

In [21]:
df['variacion_CRS_T']=df['JFK_T']-df['CRSi_T']
df['variacion_CRS_A']=df['JFK_A']-df['CRSi_A']
df['variacion_CRS_V']=df['JFK_V']-df['CRSi_V']
df['variacion_CRS_C']=df['JFK_C']-df['CRSi_C']
df['variacion_CRS_Ar']=df['JFK_Ar']-df['CRSi_Ar']
df['variacion_CRS_M']=df['JFK_M']-df['CRSi_M']
df['variacion_CRS_Ve']=df['JFK_Ve']-df['CRSi_Ve']

# Creacion dde columna Velocidad de Variación CRS

In [22]:
df['vel_var_CRS']=df['variacion_CRS_T'] / df['seguimiento'] 
df['vel_var_CRS_A']=df['variacion_CRS_A'] / df['seguimiento'] 
df['vel_var_CRS_V']=df['variacion_CRS_V'] / df['seguimiento'] 
df['vel_var_CRS_C']=df['variacion_CRS_C'] / df['seguimiento'] 
df['vel_var_CRS_Ar']=df['variacion_CRS_Ar'] / df['seguimiento'] 
df['vel_var_CRS_M']=df['variacion_CRS_M'] / df['seguimiento'] 
df['vel_var_CRS_Ve']=df['variacion_CRS_Ve'] / df['seguimiento'] 

# DF final

## Preprocesado utilizado en la colab

### Mapeo

In [23]:
df.ingreso.describe()

count                      13
unique                     12
top       2019-07-04 00:00:00
freq                        2
first     2006-06-14 00:00:00
last      2019-12-10 00:00:00
Name: ingreso, dtype: object

In [24]:
#Dict to rename cols
dict_cols= {'CRSi_A':'CRSiA', 'CRSi_V':'CRSiV','CRSi_M':'CRSiM', 'CRSi_Ve':'CRSiVe', 'CRSi_C':'CRSiC', 
            'CRSi_Ar':'CRSiAr','CRSi_T':'CRSiT'}
#Rename cols
df=df.rename(columns=dict_cols)

### Dummies

In [25]:
#Get dummies
df=pd.get_dummies(data=df, columns=['conciencia_i','genero', 'etiologia'], prefix = 'cat', drop_first=True)
df['cat_Tumor']=0

In [26]:
df.columns

Index(['NHC', 'ingreso', 'alta_prote', 'alta', 'edad', 'lesion', 'CRSiA',
       'CRSiV', 'CRSiM', 'CRSiVe', 'CRSiC', 'CRSiAr', 'CRSiT', 'JFK_T',
       'JFK_A', 'JFK_V', 'JFK_M', 'JFK_Ve', 'JFK_C', 'JFK_Ar', 'EMERGENCIA',
       'JFK_fecha', 'CRSe_A', 'CRSe_V', 'CRSe_M', 'CRSe_Ve', 'CRSe_C',
       'CRSe_Ar', 'CRSe_T', 'conciencia_e', 'edad_map', 'edad_map1', 'LOS',
       'evolucion', 'seguimiento', 'variacion_CRS_T', 'variacion_CRS_A',
       'variacion_CRS_V', 'variacion_CRS_C', 'variacion_CRS_Ar',
       'variacion_CRS_M', 'variacion_CRS_Ve', 'vel_var_CRS', 'vel_var_CRS_A',
       'vel_var_CRS_V', 'vel_var_CRS_C', 'vel_var_CRS_Ar', 'vel_var_CRS_M',
       'vel_var_CRS_Ve', 'cat_SVSR', 'cat_M', 'cat_Anoxia', 'cat_TEC',
       'cat_Tumor'],
      dtype='object')

### Seleccion de columnas

In [27]:
#df=df[['edad', 'genero', 'etiologia', 'edad_map', 'edad_map1',
 #      'LOS', 'evolucion', 'CRSi_A', 'CRSi_V', 'CRSi_M', 'CRSi_Ve', 'CRSi_C',
  #     'CRSi_Ar', 'CRSi_T', 'conciencia_i', 'JFK_A', 'JFK_V', 'JFK_M',
   #    'JFK_Ve', 'JFK_C', 'JFK_Ar', 'JFK_T', 'seguimiento', 'variacion_CRS_T',
    #   'variacion_CRS_A', 'variacion_CRS_V', 'variacion_CRS_C',
     #  'variacion_CRS_Ar', 'variacion_CRS_M', 'variacion_CRS_Ve',
      # 'vel_var_CRS', 'vel_var_CRS_A', 'vel_var_CRS_V', 'vel_var_CRS_C',
       #'vel_var_CRS_Ar', 'vel_var_CRS_M', 'vel_var_CRS_Ve', 
       #'CRSe_A', 'CRSe_V', 'CRSe_M', 'CRSe_Ve', 'CRSe_C', 'CRSe_Ar', 'CRSe_T',
       #'conciencia_e', 'EMERGENCIA']]

### Scaling

In [28]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#scaler = StandardScaler()
#y = df['EMERGENCIA']
#df = df.drop('EMERGENCIA', axis=1)
#df[:] = scaler.fit_transform(df.values)
#df['EMERGENCIA'] = y  

### Guardo el DF

In [29]:
#df.to_excel('df_held_out.xlsx')

# Validación externa del reporte preliminar

In [30]:
from sklearn.neural_network import MLPClassifier

In [31]:
#Select model
import pickle
model_selected='MLPClassifier_NF37.sav'

variables=['edad', 'edad_map', 'edad_map1', 'evolucion', 'cat_SVSR', 'cat_M','cat_Anoxia', 'cat_TEC',
                       'cat_Tumor',
                       'CRSiA', 'CRSiV', 'CRSiM','CRSiVe', 'CRSiC', 'CRSiAr', 'CRSiT',
                       'JFK_A', 'JFK_V', 'JFK_M','JFK_Ve', 'JFK_C', 'JFK_Ar', 'JFK_T',
                       'variacion_CRS_T','variacion_CRS_A', 'variacion_CRS_V', 'variacion_CRS_C','variacion_CRS_Ar',
                       'variacion_CRS_M', 'variacion_CRS_Ve',
                       'vel_var_CRS', 'vel_var_CRS_A', 'vel_var_CRS_V', 'vel_var_CRS_C','vel_var_CRS_Ar', 
                       'vel_var_CRS_M', 'vel_var_CRS_Ve']

# Load the model from disk
loaded_model = pickle.load(open(model_selected, 'rb'))

In [32]:
loaded_model

MLPClassifier(activation='relu', alpha=10, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=81, learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=150,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=42, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [33]:
SF_MLPClassifier_NF37=df[['edad', 'edad_map', 'edad_map1', 'evolucion', 'cat_SVSR', 'cat_M','cat_Anoxia', 'cat_TEC',
                       'cat_Tumor',
                       'CRSiA', 'CRSiV', 'CRSiM','CRSiVe', 'CRSiC', 'CRSiAr', 'CRSiT',
                       'JFK_A', 'JFK_V', 'JFK_M','JFK_Ve', 'JFK_C', 'JFK_Ar', 'JFK_T',
                       'variacion_CRS_T','variacion_CRS_A', 'variacion_CRS_V', 'variacion_CRS_C','variacion_CRS_Ar',
                       'variacion_CRS_M', 'variacion_CRS_Ve',
                       'vel_var_CRS', 'vel_var_CRS_A', 'vel_var_CRS_V', 'vel_var_CRS_C','vel_var_CRS_Ar', 
                       'vel_var_CRS_M', 'vel_var_CRS_Ve','EMERGENCIA']]
X_test_held_out=SF_MLPClassifier_NF37

In [34]:
from sklearn.metrics import classification_report

y_test_held_out = X_test_held_out.EMERGENCIA.values
X_test_held_out=X_test_held_out.drop(X_test_held_out.columns[len(X_test_held_out.columns)-1], axis=1).values

y_pred = loaded_model.predict(X_test_held_out)
target_names = ['No recuperacion', 'Recuperacion']
print(classification_report(y_test_held_out, y_pred, target_names=target_names))

                 precision    recall  f1-score   support

No recuperacion       0.82      1.00      0.90         9
   Recuperacion       1.00      0.50      0.67         4

       accuracy                           0.85        13
      macro avg       0.91      0.75      0.78        13
   weighted avg       0.87      0.85      0.83        13



In [35]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test_held_out, y_pred)

0.75